In [236]:
import shlex
import pandas as pd
import numpy as np



In [57]:
text_data = pd.read_csv('clinical_trials_dataset.csv')
#text_data = text_data.dropna(axis=0,how='any').reset_index(drop=True)
text_data = text_data[['OfficialTitle','StudyType','StartDate',
           'Condition','ConditionAncestorTerm',
           'ArmGroupInterventionName','InterventionArmGroupLabel','InterventionDescription']].copy()
#text_data[text_data['ArmGroupInterventionName'].str.contains('Genetic')]
text_data.isna().sum()

OfficialTitle                  53
StudyType                       0
StartDate                      42
Condition                       0
ConditionAncestorTerm         571
ArmGroupInterventionName     1128
InterventionArmGroupLabel    1124
InterventionDescription      1158
dtype: int64

In [110]:
#deal with nan values column by column
test = text_data.copy()
test = test.fillna({'StartDate':'unknown'}).copy()
test = test.fillna({'OfficialTitle':'unknown'}).copy()
test['ConditionAncestorTerm'] = test['ConditionAncestorTerm'].fillna(test['Condition'])
test = test.fillna({'ArmGroupInterventionName':'other or NA'}).copy()
test = test.fillna({'InterventionArmGroupLabel':'other or NA'}).copy()
test = test.fillna({'InterventionDescription':'other or NA'}).copy()
text_data = test.copy()

del test

In [112]:
#replace nan values with other appropriate values
text_data.isna().sum()

OfficialTitle                0
StudyType                    0
StartDate                    0
Condition                    0
ConditionAncestorTerm        0
ArmGroupInterventionName     0
InterventionArmGroupLabel    0
InterventionDescription      0
dtype: int64

In [136]:
text_data[(text_data['ArmGroupInterventionName'].str.contains('Genetic:')) & (text_data['StudyType']=='Interventional')]

,OfficialTitle,StudyType,StartDate,Condition,ConditionAncestorTerm,ArmGroupInterventionName,InterventionArmGroupLabel,InterventionDescription
0,"Phase I/IIa, First-in-human, Open-label, Singl...",Interventional,September 2024,Drug Resistant Epilepsy,Brain Diseases|Central Nervous System Diseases...,Genetic: lentiviral gene therapy,lentiviral gene therapy treatment (Interventio...,lentiviral gene therapy to treat drug resistan...
5,"A Phase I Study to Assess the Safety, Tolerabi...",Interventional,"February 7, 2022",Alzheimer's Disease|Mild Cognitive Impairment,Dementia|Brain Diseases|Central Nervous System...,Genetic: AAV2-BDNF Gene Therapy|Biological: AA...,Gene transfer of AAV2-BDNF|Gene transfer of AA...,AAV2-BDNF is a genetically engineered adeno-as...
7,A Phase I/II Clinical Trial of Hematopoietic S...,Interventional,"April 9, 2010",Lysosomal Storage Disease|Metachromatic Leukod...,"Metabolism, Inborn Errors|Genetic Diseases, In...",Genetic: OTL-200 Gene Therapy,OTL-200 Gene Therapy,Autologous hematopoietic stem/progenitor cells...
8,Phase I/II Clinical Trial of Autologous Hemato...,Interventional,"July 23, 2021",Severe Combined Immunodeficiency Due to RAG1 D...,Immune System Diseases|Primary Immunodeficienc...,Genetic: Gene therapy,Gene therapy,Patients will be infused with autologous CD34+...
12,A Phase I/II Gene Therapy Trial for X-CGD With...,Interventional,July 2013,X-linked Chronic Granulomatous Disease,Pathologic Processes|Phagocyte Bactericidal Dy...,Genetic: ex-vivo gene-therapy,ex-vivo gene-therapy,"transplantation autologous CD34+ cells, transd..."
...,...,...,...,...,...,...,...,...
5804,"An Open-label, Dose-escalation Study to Evalua...",Interventional,"May 5, 2023",Neovascular Age-related Macular Degeneration,Retinal Degeneration|Retinal Diseases|Eye Dise...,Genetic: SKG0106|Genetic: SKG0106|Genetic: SKG...,Dose Level 1|Dose Level 2|Dose Level 3,SKG0106 is a recombinant adeno-associated viru...
5858,WEST-KOaST Study: WES (Whole Exome Sequencing)...,Interventional,"December 1, 2023",Testicular Cancer,Endocrine Gland Neoplasms|Neoplasms by Site|Ne...,Genetic: Analysis of biological samples of sal...,First-degree family members of patients with t...,The project aims to collect biological samples...
5864,Phase II Study of Capecitabine in Combination ...,Interventional,November 2006,Esophageal Cancer,Gastrointestinal Neoplasms|Digestive System Ne...,Drug: capecitabine|Drug: oxaliplatin|Genetic: ...,Chemo|Chemo|Chemo|Chemo|Chemo|Chemo|Chemo|Chem...,Oral|IV|Correlative Study|Correlative Study|Co...
5887,Neoadjuvant Intratumoral Injection of Dendriti...,Interventional,"May 1, 2006",Breast Cancer,Neoplasms by Site|Neoplasms|Breast Diseases|Sk...,Biological: therapeutic autologous dendritic c...,Vaccine|Vaccine|Vaccine|Vaccine|Vaccine|Vaccin...,injected into the primary breast mass or palpa...


In [ ]:
#this is for condition names
#the split('|') must be already applied to cond_list, function does not do that
def minimum_cond_name(cond_list):
    con_size=[]
    for elem in text_data['ConditionAncestorTerm'][0].split('|'):
        con_size.append(elem.count(' '))
    return cond_list[con_size.index(min(con_size))]

In [240]:
#add column with modified conditions
text_data['mod_ConditionAncestorTerm'] = [minimum_cond_name(con_term.split('|')) for con_term in text_data['ConditionAncestorTerm']]

In [286]:
text_data[text_data['mod_ConditionAncestorTerm'].str.contains('Anemia')]

,OfficialTitle,StudyType,StartDate,Condition,ConditionAncestorTerm,ArmGroupInterventionName,InterventionArmGroupLabel,InterventionDescription,mod_ConditionAncestorTerm
1,"Growth and Development, Health-related Quality...",Observational,"June 5, 2023",Transfusion-dependent Beta-Thalassemia|Gene Th...,"Anemia, Hemolytic, Congenital|Anemia, Hemolyti...",Genetic: Gene therapy,Gene therapy group,Autologous edited hematopoietic stem cell tran...,"Anemia, Hemolytic, Congenital"
22,Gene Therapy Communication: Use of a Needs Ass...,Observational,"June 1, 2023",Sickle Cell Disease,"Anemia, Hemolytic, Congenital|Anemia, Hemolyti...",Other: Interview|Other: Interview|Other: Inter...,Healthcare Worker|Parent/caregiver|Young Adult,The following three groups of stakeholders wil...,"Anemia, Hemolytic, Congenital"
48,Gene Therapy for Pyruvate Kinase Deficiency (P...,Interventional,"July 6, 2020",Pyruvate Kinase Deficiency,"Anemia, Hemolytic, Congenital|Anemia, Hemolyti...",Biological: RP-L301,RP-L301,Autologous genetically modified CD34+ hematopo...,"Anemia, Hemolytic, Congenital"
95,Long-term Follow-up of Subjects With Sickle Ce...,Observational,"October 21, 2020",Sickle Cell Disease,"Anemia, Hemolytic, Congenital|Anemia, Hemolyti...",Other: Safety and efficacy assessments,Subjects with sickle-cell disease,"Safety evaluations, disease-specific assessmen...","Anemia, Hemolytic, Congenital"
97,Long-term Follow-up of Subjects With Transfusi...,Observational,January 2014,Transfusion-dependent Beta-Thalassemia,"Anemia, Hemolytic, Congenital|Anemia, Hemolyti...",Other: Safety and efficacy assessments,Subjects with Transfusion-Dependent β-Thalassemia,"Vector copy number (VCN) measurement, safety e...","Anemia, Hemolytic, Congenital"
...,...,...,...,...,...,...,...,...,...
4951,an Open Label Trial of Evaluation of the Safet...,Interventional,"December 25, 2023",Thalassemia Major,"Anemia, Hemolytic, Congenital|Anemia, Hemolyti...",Biological: γ-globin reactivated autologous he...,γ-globin reactivated autologous hematopoietic ...,gene edited autologous hematopoietic stem cell...,"Anemia, Hemolytic, Congenital"
5099,Genes Influencing Iron Overload State,Observational,"September 21, 2010",Sickle Cell Disease|Thalassemia|Marrow Aplasia,"Anemia, Hemolytic, Congenital|Anemia, Hemolyti...",other or NA,other or NA,other or NA,"Anemia, Hemolytic, Congenital"
5450,A Phase 1/2/3 Study of the Safety and Efficacy...,Interventional,"September 14, 2018","Beta-Thalassemia|Thalassemia|Genetic Diseases,...","Anemia, Hemolytic, Congenital|Anemia, Hemolyti...",Biological: CTX001,CTX001,Administered by IV infusion following myeloabl...,"Anemia, Hemolytic, Congenital"
5578,Detection of RNA Binding Protein Pumilio1 (PUM...,Observational,August 2023,"Sickle Cell Disease, Beta Thalassemia Intermedia","Anemia|Hematologic Diseases|Anemia, Hemolytic,...",Diagnostic Test: RNA Binding Protein Pumilio1 ...,Control group|thalassemia intermedia patients ...,Quantatative Real -Time PCR for quantification...,Anemia


In [273]:
condword_list = [' '.join(word.split(',')) if ',' in word else word for word in text_data['mod_ConditionAncestorTerm'].values]
cond_word_count = []
for word in condword_list:
    for each_word in word.split(' '):
        cond_word_count.append(each_word)

frequency_map = {}
for item in cond_word_count:
    if item in frequency_map:
        frequency_map[item] += 1
    else:
        frequency_map[item] = 1

frequency_map_df = pd.DataFrame(frequency_map.items(),columns=['word','frequency']).sort_values(by='frequency',ascending=False)

In [288]:
pick_words= ['Neoplasms','Diseases','Disorders','Disease','Carcinoma','Infections','Anemia']
frequency_map_df

,word,frequency
8,Neoplasms,2204
3,,1330
1,Diseases,942
27,by,914
76,Type,521
...,...,...
518,Metastasis,1
515,Frontline,1
514,From,1
512,Complete,1


In [294]:
for word in pick_words:
    if word in ''.join(text_data['mod_ConditionAncestorTerm'][0]):
        print(word)

Diseases
Disease


False

In [233]:
modified_armgroup_interventionName=[]
for elem_arm in text_data['ArmGroupInterventionName']:
    if 'Genetic:' in elem_arm:
        if '|' in elem_arm:
            for elem_arm_gene in elem_arm.split('|'):
                if 'Genetic:' in elem_arm_gene and ('therapy' in elem_arm_gene.lower() or 'vector' in elem_arm_gene.lower()):
                    modified_armgroup_interventionName.append(elem_arm_gene)
                elif 'Genetic:' in elem_arm_gene:
                    modified_armgroup_interventionName.append('other genetic')
                    print(elem_arm)
        elif '|' not in elem_arm and ('therapy' in elem_arm.lower() or 'vector' in elem_arm.lower()):
            modified_armgroup_interventionName.append(elem_arm)
        else:
            print(elem_arm)
            modified_armgroup_interventionName.append('other genetic')
    else:
        modified_armgroup_interventionName.append('other') 


Genetic: gene transduced PBL and/or gene transduced HSC
Genetic: rAAV1.tMCK.human-alpha-sarcoglycan- First cohort|Genetic: Genetic: rAAV1.tMCK.human-alpha-sarcoglycan- Second cohort
Genetic: rAAV1.tMCK.human-alpha-sarcoglycan- First cohort|Genetic: Genetic: rAAV1.tMCK.human-alpha-sarcoglycan- Second cohort
Genetic: OTL-103
Genetic: Strimvelis
Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314
Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314
Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314
Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314
Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314
Genetic: Ad5-yCD/mutTKSR39rep-ADP
Genetic: Intervention on primary cultured cells
Genetic: OTL-103
Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314
Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX-314|Genetic: RGX

In [234]:
np.unique(np.array(modified_armgroup_interventionName),return_counts=True)

(array(['Genetic: AAV directed hLDLR gene therapy',
        'Genetic: AAV2-BDNF Gene Therapy', 'Genetic: AB-1001 Gene Therapy',
        'Genetic: Autologous CD34 positive cells transduced with a lentiviral vector containing human WAS gene',
        'Genetic: Autologous CD4+ T cells genetically modified with a retroviral vector expressing the MazF endoribonuclease gene (MazF-T), given via intravenous infusion.',
        'Genetic: Autologous TRICAR-ALL T-cells and lymphodepletion chemotherapy',
        'Genetic: Autologous hematopoietic stem cells genetically modified with GLOBE lentiviral vector encoding for the human beta-globin gene',
        'Genetic: Frozen autologous CD34+ hematopoietic stem and progenitor cells genetically modified with the lentiviral vector IDUA LV, encoding for the α-L-iduronidase cDNA, in their final formulation medium.',
        'Genetic: Gene Therapy',
        'Genetic: Gene Therapy product CYL-02 = plasmid DNA pre-complexed to linear polyethylenimine encodin

In [173]:
minimum_cond_name(text_data['ConditionAncestorTerm'][3].split('|'))

'Immune System Diseases'